# MetricsTracker Demo (TT-31)

Demonstrates the delta-1 position metrics engine joining live DXLink Quote data
with account positions. Loads positions via REST, subscribes to DXLink for
streamer symbols, and displays the joined DataFrame with theoretical Greeks.

In [ ]:
import asyncio
import logging
import os

import pandas as pd
from dotenv import load_dotenv
from IPython.display import Markdown, display

from tastytrade.accounts import AccountsClient
from tastytrade.analytics.metrics import DELTA_1_TYPES, OPTION_TYPES, MetricsTracker
from tastytrade.config import RedisConfigManager
from tastytrade.config.enumerations import Channels
from tastytrade.connections import Credentials
from tastytrade.connections.requests import AsyncSessionHandler
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.messaging.processors.metrics import MetricsEventProcessor

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", 40)

load_dotenv("/workspace/.env", override=True)

logging.basicConfig(level=logging.INFO)
logging.getLogger("websockets").setLevel(logging.WARNING)

_OBFUSCATE = os.getenv("OBFUSCATE_ACCOUNTS", "false").lower() == "true"


def mask(account_number: str) -> str:
    if not _OBFUSCATE or len(account_number) <= 4:
        return account_number
    return "***" + account_number[-4:]


print(f"Account obfuscation: {'ON' if _OBFUSCATE else 'OFF'}")

# 1. Connect & Fetch Positions

In [ ]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize(force=True)

credentials = Credentials(config=config, env="Live")
session = await AsyncSessionHandler.create(credentials)
client = AccountsClient(session)

account = credentials.account_number
positions = await client.get_positions(account)

display(Markdown(f"**{len(positions)} positions** in account {mask(account)}"))

pos_summary = [{
    "Symbol": p.symbol,
    "Type": p.instrument_type.value,
    "Qty": p.quantity,
    "Direction": p.quantity_direction.value,
    "Streamer": p.streamer_symbol,
} for p in positions]
display(pd.DataFrame(pos_summary))

# 2. Initialize MetricsTracker

In [ ]:
tracker = MetricsTracker()
tracker.load_positions(positions)

streamer_symbols = tracker.get_streamer_symbols()
skipped = [p.symbol for p in positions if p.streamer_symbol is None]

display(Markdown(f"**Tracking {len(tracker.securities)} securities** ({len(skipped)} skipped — no streamer symbol)"))
if skipped:
    print(f"Skipped: {skipped}")

display(Markdown("**Initial state (no market data yet):**"))
display(tracker.df[["symbol", "instrument_type", "quantity", "quantity_direction",
                     "bid_price", "ask_price", "mid_price",
                     "delta", "gamma", "theta", "vega"]])

# 3. Open DXLink & Attach MetricsEventProcessor

In [ ]:
dxlink = DXLinkManager()
await dxlink.open(credentials=credentials)

# Attach MetricsEventProcessor to the Quote handler only
router = dxlink.router
if router is None:
    raise RuntimeError("DXLink router not initialized")

quote_handler = router.handler.get(Channels.Quote)
if quote_handler is None:
    raise RuntimeError("Quote handler not found")

processor = MetricsEventProcessor(tracker)
quote_handler.add_processor(processor)

print(f"MetricsEventProcessor attached to Quote handler")
print(f"Quote handler processors: {list(quote_handler.processors.keys())}")

# 4. Subscribe to Position Streamer Symbols

In [ ]:
symbols_list = sorted(streamer_symbols)
print(f"Subscribing to {len(symbols_list)} symbols:")
for s in symbols_list:
    print(f"  {s}")

await dxlink.subscribe(symbols_list)
print(f"\nSubscribed. Waiting for quotes to flow...")

# 5. View Live Metrics

Re-run this cell to see updated prices as quotes arrive.

In [ ]:
await asyncio.sleep(3)  # allow quotes to arrive

df = tracker.df

# Reorder columns for readability
display_cols = [
    "symbol", "instrument_type", "quantity", "quantity_direction",
    "bid_price", "ask_price", "mid_price",
    "delta", "gamma", "theta", "vega", "implied_volatility",
    "price_updated_at",
]
available_cols = [c for c in display_cols if c in df.columns]

populated = df["bid_price"].notna().sum()
display(Markdown(f"**{populated}/{len(df)} securities** have live quote data"))
display(df[available_cols])

# 6. Delta-1 vs Options Greeks Comparison

Shows that delta-1 products have theoretical Greeks while options show None.

In [ ]:
df = tracker.df

delta1_mask = df["instrument_type"].isin([t.value for t in DELTA_1_TYPES])
option_mask = df["instrument_type"].isin([t.value for t in OPTION_TYPES])

greeks_cols = ["symbol", "instrument_type", "delta", "gamma", "theta", "vega", "implied_volatility"]

display(Markdown("**Delta-1 positions** (theoretical Greeks):"))
display(df.loc[delta1_mask, greeks_cols])

display(Markdown("**Option positions** (Greeks = None, awaiting TT-37 GreeksEvent):"))
display(df.loc[option_mask, greeks_cols])

# 7. Cleanup

In [ ]:
await dxlink.close()
await session.close()
print("All connections closed.")